In [285]:
import os
import pandas as pd
import geopandas as gpd 
from shapely.geometry import Point
import numpy as np

In [286]:
osm_impots = './raw/osm_impots.geojson'
osm_impots_gdf=gpd.read_file(osm_impots)
osm_impots_gdf.head()

,osm_id,code,name,geometry
0,26617684,1500,Direction des impôts,POINT (4.03655 36.71474)
1,153107861,1500,Receveur des impots - Dely brahim,POINT (2.98892 36.74876)
2,185615561,1500,Lagr.ilot 24-1 Les impôts,POINT (5.41145 36.18817)
3,234050287,1500,Inspection des Impots مفتشية الضرائب وادي النجاء,POINT (6.11954 36.42813)
4,240332545,1500,Service des Impôts,POINT (3.03324 36.73448)


In [287]:
# Load the targeted dataset
impots_gdf = gpd.read_file('./impots_to_check.geojson')
impots_gdf.head()

,address,lat,lon,tel,name,wilaya,to_check,code,geometry
0,"ADRAR,Boulevard Mohamed El Atchane,Adrar",28.017440,-0.264250,049 36 41 18 - 049 36 41 23 - 049 36 41 24 ...,. IMPOTS Direction,Adrar,False,1,POINT (-0.26425 28.01744)
1,"ADRAR,Rue Taleb Mohamed,Adrar",27.876660,-0.270800,49361062,. IMPOTS Direction,Adrar,False,1,POINT (-0.27080 27.87666)
2,"ADRAR,Route 5 Juillet 1962,Adrar",28.033886,1.659626,49364129,. IMPOTS Direction,Adrar,False,1,POINT (1.65963 28.03389)
3,"ADRAR,Adrar",27.876660,-0.270800,49364113,. IMPOTS Direction,Adrar,False,1,POINT (-0.27080 27.87666)
4,"3,REGGANE,Cité El Kaous Labyad,Adrar",26.724265,0.165729,49320202,. IMPOTS Direction,Adrar,False,1,POINT (0.16573 26.72427)


In [288]:
# Define source CRS to geographic and project to a Pseudo-Mercator
# Useful for spatial analysis (buffer, intersect, ...)

impots_gdf = impots_gdf.set_crs("EPSG:4326")
impots_gdf= impots_gdf.to_crs("EPSG:3857")

osm_impots_gdf = osm_impots_gdf.set_crs("EPSG:4326")
osm_impots_gdf = osm_impots_gdf.to_crs("EPSG:3857")

osm_impots_gdf.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [289]:
impots_gdf.reset_index(inplace=True)
impots_gdf = impots_gdf.rename(columns = {'index':'num'})

In [290]:
impots_gdf['buffer'] = impots_gdf.geometry.buffer(500)
buffers = impots_gdf[['num', 'buffer']]
buffers = buffers.rename(columns={'buffer': 'geometry'})
buffers = gpd.GeoDataFrame(buffers)
buffers = buffers.set_crs("EPSG:3857")
gpd.GeoDataFrame(buffers).to_file("buffers.geojson", driver='GeoJSON')
buffers.head()

,num,geometry
0,0,"POLYGON ((-28916.142 3251172.790, -28918.550 3..."
1,1,"POLYGON ((-29645.296 3233432.306, -29647.703 3..."
2,2,"POLYGON ((185248.721 3253246.711, 185246.314 3..."
3,3,"POLYGON ((-29645.296 3233432.306, -29647.703 3..."
4,4,"POLYGON ((18948.857 3089064.389, 18946.449 308..."


In [291]:
osm_impots_gdf

,osm_id,code,name,geometry
0,26617684,1500,Direction des impôts,POINT (449346.831 4399419.361)
1,153107861,1500,Receveur des impots - Dely brahim,POINT (332724.662 4404144.765)
2,185615561,1500,Lagr.ilot 24-1 Les impôts,POINT (602399.661 4326544.165)
3,234050287,1500,Inspection des Impots مفتشية الضرائب وادي النجاء,POINT (681224.537 4359693.075)
4,240332545,1500,Service des Impôts,POINT (337658.246 4402160.736)
...,...,...,...,...
58,582387496,1500,المحافظة العقارية - مديرية الضرائب - املاك الدولة,POINT (828295.001 4363359.715)
59,744547188,1500,مركز الضرائب,POINT (739881.724 4362027.811)
60,196930892,2099,Inspection des impôts,POINT (707063.495 4198865.890)
61,552216646,2099,ISPECTION DES IMPOT,POINT (375067.273 4394953.941)


In [292]:
join= gpd.sjoin(osm_impots_gdf,buffers,how="left",op="within")
join.drop(columns=['code'],inplace=True)
found = join[~join['num'].isna()].copy()
found['num'] = found['num'].astype(int)
found.drop(columns=['index_right'],inplace=True)

not_found = join[join['num'].isna()].copy()
not_found.drop(columns=['num','index_right'],inplace=True)

In [293]:
impots_gdf['osm_intersect']=False

In [294]:
impots_gdf.loc[impots_gdf.num.isin(found.num),'osm_intersect'] = True

In [295]:
home_path = os.getenv('HOME')
wilayas_shp= home_path+'/GeoData/Others/LimAdmin/wilayas.shp'
wilayas_gdf = gpd.read_file(wilayas_shp)
wilayas_gdf.rename(columns={"CODE": "code"}, inplace=True)
wilayas_gdf.rename(columns={"WILAYA": "wilaya"}, inplace=True)
wilayas_gdf = wilayas_gdf.to_crs("EPSG:3857")
not_found = gpd.sjoin(not_found,wilayas_gdf,op="within")

In [296]:
new_points = not_found.drop(columns=['index_right','osm_id'])

In [297]:
# Load osm_building Features and calculate polygon centroids
osm_building_shp = home_path+'/GeoData/OSM/algeria-latest-free.shp/gis_osm_buildings_a_free_1.shp'
osm_building_gdf = gpd.read_file(osm_building_shp)
osm_building_gdf.head()

,osm_id,code,fclass,name,type,geometry
0,23453590,1500,building,None,None,"POLYGON ((3.28630 36.72337, 3.28680 36.72384, ..."
1,23453621,1500,building,None,industrial,"POLYGON ((3.28839 36.72555, 3.28945 36.72653, ..."
2,23453649,1500,building,None,industrial,"POLYGON ((3.29652 36.73153, 3.29695 36.73203, ..."
3,26224008,1500,building,Hilton Alger,hotel,"POLYGON ((3.15595 36.73836, 3.15634 36.73856, ..."
4,26224288,1500,building,Crystal Lounge,None,"POLYGON ((3.15561 36.73811, 3.15583 36.73823, ..."


In [298]:
osm_building_gdf = osm_building_gdf.set_crs("EPSG:4326")
osm_building_gdf = osm_building_gdf.to_crs("EPSG:3857")

osm_building_gdf.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [299]:
res_intersection = gpd.overlay(buffers, osm_building_gdf, how='intersection')

In [300]:
res_intersection.head()

,num,osm_id,code,fclass,name,type,geometry
0,1,304696040,1500,building,Direction des Foret,None,"POLYGON ((-30008.217 3233910.080, -29980.165 3..."
1,3,304696040,1500,building,Direction des Foret,None,"POLYGON ((-30008.217 3233910.080, -29980.165 3..."
2,1,432509512,1500,building,Algérie poste,None,"POLYGON ((-30623.766 3233287.163, -30626.922 3..."
3,3,432509512,1500,building,Algérie poste,None,"POLYGON ((-30623.766 3233287.163, -30626.922 3..."
4,1,432511940,1500,building,None,None,"POLYGON ((-30635.688 3233334.760, -30639.082 3..."


In [301]:
res = res_intersection[['num', 'geometry']]
res.to_file("intersection.geojson", driver='GeoJSON')

In [302]:
res_grouped = res [['num']]
res_grouped = res_grouped.groupby('num').count()
res_grouped['to_check'] = False
res_grouped.count()

to_check    786
dtype: int64

In [303]:
impots_to_check_gdf = impots_gdf.merge(res_grouped,left_on='num',right_on='num',how='left')
impots_to_check_gdf.count()

num              897
address          897
lat              897
lon              897
tel              897
name             897
wilaya           897
to_check_x       897
code             897
geometry         897
buffer           897
osm_intersect    897
to_check_y       786
dtype: int64

In [304]:
impots_to_check_gdf['to_check_y'].fillna(True,inplace=True)

In [305]:
impots_to_check_gdf['to_check'] = impots_to_check_gdf['to_check_x'] | impots_to_check_gdf['to_check_y']

In [307]:
impots_to_check_gdf

928

In [309]:
impots= impots_to_check_gdf.append(new_points,ignore_index=True)

In [310]:
impots

,num,address,lat,lon,tel,name,wilaya,to_check_x,code,geometry,buffer,osm_intersect,to_check_y,to_check
0,0.0,"ADRAR,Boulevard Mohamed El Atchane,Adrar",28.017440,-0.264250,049 36 41 18 - 049 36 41 23 - 049 36 41 24 ...,. IMPOTS Direction,Adrar,False,1,POINT (-29416.142 3251172.790),"POLYGON ((-28916.142 3251172.790, -28918.550 3...",False,True,True
1,1.0,"ADRAR,Rue Taleb Mohamed,Adrar",27.876660,-0.270800,49361062,. IMPOTS Direction,Adrar,False,1,POINT (-30145.296 3233432.306),"POLYGON ((-29645.296 3233432.306, -29647.703 3...",False,False,False
2,2.0,"ADRAR,Route 5 Juillet 1962,Adrar",28.033886,1.659626,49364129,. IMPOTS Direction,Adrar,False,1,POINT (184748.721 3253246.711),"POLYGON ((185248.721 3253246.711, 185246.314 3...",False,True,True
3,3.0,"ADRAR,Adrar",27.876660,-0.270800,49364113,. IMPOTS Direction,Adrar,False,1,POINT (-30145.296 3233432.306),"POLYGON ((-29645.296 3233432.306, -29647.703 3...",False,False,False
4,4.0,"3,REGGANE,Cité El Kaous Labyad,Adrar",26.724265,0.165729,49320202,. IMPOTS Direction,Adrar,False,1,POINT (18448.857 3089064.389),"POLYGON ((18948.857 3089064.389, 18946.449 308...",False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,NaN,NaN,NaN,NaN,NaN,Centre des impôts,MOSTAGANEM,NaN,27,POINT (8590.708 4289878.432),None,NaN,NaN,NaN
924,NaN,NaN,NaN,NaN,NaN,Centre des Impôts,LAGHOUAT,NaN,3,POINT (317636.649 4001375.485),None,NaN,NaN,NaN
925,NaN,NaN,NaN,NaN,NaN,مديرية الضرائب-سيدي خويلد,OUARGLA,NaN,30,POINT (599858.403 3755638.043),None,NaN,NaN,NaN
926,NaN,NaN,NaN,NaN,NaN,Centre des impôts de Boufarik,BLIDA,NaN,9,POINT (323397.848 4379900.365),None,NaN,NaN,NaN


In [311]:
impots.drop(columns=['num'],inplace=True)
impots.reset_index(inplace=True)
impots = impots.rename(columns = {'index':'num'})

In [316]:
impots.drop(columns=['to_check_x','to_check_y','buffer'],inplace=True)

In [321]:
#impots = impots.set_crs("EPSG:3857")
impots = impots.to_crs("EPSG:4326")
impots.to_file('impots_to_check_osm.geojson',driver='GeoJSON')